In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
!pip install pytorch-pretrained-bert pytorch-nlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pad_sequences

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla T4'

In [ ]:
# Tải lên file huấn luyện từ máy của bạn
from google.colab import files
uploaded = files.upload()

Saving data.csv to data (3).csv


In [ ]:
df = pd.read_csv("/content/data.csv", delimiter=',', header=None, names=['label', 'sentence'])
df.shape
df

,label,sentence
0,1,Có tôi là Phúc
1,0,Không tôi đang đang bận
2,1,Có tôi có thể giúp bạn
3,1,Có tôi sẽ cố gắng hỗ trợ
4,0,Không tôi không có thời gian
5,1,Có tôi sẽ xem xét yêu cầu của bạn
6,0,Không tôi không thể làm được
7,0,Không tôi đã có lịch trình khác
8,1,Có tôi sẽ liên hệ lại với bạn
9,1,Có tôi rất vui được giúp bạn


In [ ]:
# Tạo danh sách câu và nhãn
sentences = df.sentence.values

# Chúng ta cần thêm các token đặc biệt ở đầu và cuối mỗi câu để BERT hoạt động chính xác
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
labels = df.label.values

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])

Tokenize the first sentence:
['[CLS]', 'co', 'to', '##i', 'la', 'ph', '##uc', '[SEP]']


In [ ]:
# Đặt độ dài chuỗi tối đa. Chuỗi dài nhất trong tập huấn luyện của chúng ta là 47, nhưng dù sao chúng ta cần chừa khoảng trống ở cuối.
# Trong bài báo gốc, các tác giả đã sử dụng độ dài 512.
MAX_LEN = 128
# Sử dụng bộ tách từ BERT để chuyển đổi token thành index của chúng trong bộ từ vựng BERT
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
# Đệm thêm vào các chuỗi đầu vào
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
# Tạo mảng chứa các attention mask
attention_masks = []

# Tạo mask chứa các giá trị 1 cho mỗi koten và 0 cho các giá trị đệm
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)
  # Sử dụng train_test_split để chia dữ liệu của ta thành tập huấn luyện và tập xác nhận

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, random_state=2018, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids, random_state=2018, test_size=0.1)
# Chuyển dữ liệu về dạng tensor của pytorch, đây là kiểu dữ liệu bắt buộc cho mô hình của chúng ta

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)
# lựa chọn batch size sử dụng huấn luyện. Với fine-tuning BERT, các tác giả khuyến nghị đặt giá trị này là 16 hoặc 32
batch_size = 32

# Tạo iterator với DataLoader. Điều này sẽ giúp tiết kiệm bộ nhớ khi huấn luyện. Khác với vòng lặp, với một iterator thì toàn bộ dataset sẽ không cần phải nạp lên bộ nhớ

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [ ]:
# Nạp BertForSequenceClassification, mô hình BERT được huấn luyện sẵn với một lớp phân loại ở trên cùng

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=Fa

In [ ]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

In [ ]:
# Biến này chứa tất cả thông tin về siêu tham số cần cho huấn luyện
optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=2e-5,
                     warmup=.1)

In [ ]:
# Hàm để tính độ chính xác giữa kết quả dự đoán và nhãn thực
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
# Lưu giá trị loss và độ chính xác để vẽ biểu đồ
train_loss_set = []

# Số lượng epoch huấn luyện (các tác giả khuyến nghị giá trị từ 2 đến 4)
epochs = 4

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):
  
  
  # Huấn luyện
  
  # Đưa mô hình về chế độ huấn luyện
  model.train()
  
  # Các biến theo dõi
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  
  # Huấn luyện trong 1 epoch
  for step, batch in enumerate(train_dataloader):
    # Đưa dữ liệu vào GPU
    batch = tuple(t.to(device) for t in batch)
    # Lấy dữ liệu đầu vào từ dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Xóa gradient
    optimizer.zero_grad()
    # Truyền dữ liệu qua mạng
    loss = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    train_loss_set.append(loss.item())    
    # Lan truền ngược
    loss.backward()
    # Cập nhật trọng số mạng
    optimizer.step()
    
    
    # Cập nhật biến theo dõi
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))
    
    
  # Xác nhận

  # Đặt mô hình vào chế độ đánh giá để đánh giá lỗi trên tập xác nhận
  model.eval()

  # Các biến theo dõi 
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0

  # Đánh giá trong 1 epoch
  for batch in validation_dataloader:
    # Đưa vào GPU
    batch = tuple(t.to(device) for t in batch)
    # Lấy dữ liệu đầu vào từ dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Yêu cầu dữ liệu không tính gradient, tiết kiệm bộ nhớ và tăng tốc độ
    with torch.no_grad():
      # Chiều xuôi, tính toán dự toán 
      logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    
    # Đưa giá trị dự đoán và nhãn về CPU để tính độ chính xác
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

  print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

Epoch:  25%|██▌       | 1/4 [00:00<00:02,  1.11it/s]

Train loss: 0.7055394649505615
Validation Accuracy: 0.75


Epoch:  50%|█████     | 2/4 [00:01<00:01,  1.20it/s]

Train loss: 0.6327571272850037
Validation Accuracy: 1.0


Epoch:  75%|███████▌  | 3/4 [00:02<00:00,  1.21it/s]

Train loss: 0.5475683808326721
Validation Accuracy: 1.0


Epoch: 100%|██████████| 4/4 [00:03<00:00,  1.20it/s]

Train loss: 0.42983290553092957
Validation Accuracy: 1.0


In [ ]:
# Tải file chứa tập dữ liệu test lên
from google.colab import files
uploaded = files.upload()

Saving data.csv to data (4).csv


In [ ]:
df = pd.read_csv("/content/data.csv", delimiter=',', header=None, names=['label', 'sentence'])

# Danh sách câu và nhãn
sentences = df.sentence.values

# Thêm token đặc biệt vào đầu vào cuối câu để BERT hoạt động đúng
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
labels = df.label.values

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]


MAX_LEN = 128
# Sử dụng bộ tách từ của BERT để chuyển token thành số định danh trong tập từ điển của BERT
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
# Đệm thêm vào đầu vào
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
# Tạo attention masks
attention_masks = []

for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask) 

prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels)
  
batch_size = 32  
prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

FileNotFoundError: ignored

In [ ]:
# Dự đoán trên tập test

# Đưa mô hình vào chế độ đánh giá
model.eval()

# Các biến theo dõi
predictions , true_labels = [], []

# Dự đoán
for batch in prediction_dataloader:
  # Đưa vào GPU
  batch = tuple(t.to(device) for t in batch)
  # Lấy dữ liệu đầu vào từ dataloader
  b_input_ids, b_input_mask, b_labels = batch
  # Yêu cầu mô hình không tính gradient, tiết kiệm bộ nhớ và tăng tốc
  with torch.no_grad():
    # Truyefn dữ liệu, tính toán giá trị dự đoán
    logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

  # Đưa giá trị dự đoán và nhãn thật về CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Lưu giá trị dự đoán và nhãn thật
  predictions.append(logits)
  true_labels.append(label_ids)

In [ ]:
# Gọi và sử dụng thư viện matthews_corrcoef để đánh giá 
from sklearn.metrics import matthews_corrcoef
matthews_set = []

for i in range(len(true_labels)):
  matthews = matthews_corrcoef(true_labels[i],
                 np.argmax(predictions[i], axis=1).flatten())
  matthews_set.append(matthews)

In [ ]:
matthews_set

[1.0, 0.0]

In [ ]:
# Duỗi giá trị dự đoán và nhãn thật để sử dụng độ đo Matthew trên toàn bộ dataset
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
flat_true_labels = [item for sublist in true_labels for item in sublist]

In [ ]:
matthews_corrcoef(flat_true_labels, flat_predictions)

1.0

In [ ]:
pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
input_text = "Có anh hỗ trợ được"
inputs = tokenizer(input_text, truncation=True, padding=True, return_tensors="pt").to(device)
# Perform the prediction
with torch.no_grad():
    outputs = model(**inputs)

logits = outputs[0]  # Access the logits tensor directly
predicted_class = logits.argmax(dim=-1).item()
print(predicted_class)


1


In [ ]:
output_path = '/content/result/voicebot.bin'
torch.save(model.state_dict(), output_path)


In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model.load_state_dict(torch.load('/content/result/voicebot.bin'))
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
input_text = "không anh không rảnh"
inputs = tokenizer(input_text, truncation=True, padding=True, return_tensors="pt").to(device)
# Perform the prediction
with torch.no_grad():
    outputs = model(**inputs)

logits = outputs[0]  # Access the logits tensor directly
predicted_class = logits.argmax(dim=-1).item()
print(predicted_class)

0
